In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [2]:
# functions for calculating ELOs
def mov_multiplyer(elo_winner, elo_loser, score_winner, score_loser):
    mov = abs(score_winner-score_loser)
    elo_diff = elo_winner-elo_loser
    return ((mov+3)**0.8)/(7.5+0.006*(elo_diff))

def pr_winner_win(elo_winner, elo_loser):
    elo_diff = elo_loser - elo_winner
    return 1/(1+10**(elo_diff/400))

def post_elo_home_winner(elo_winner, elo_loser, score_winner, score_loser):
    return (elo_winner-100) + 20*(1-pr_winner_win(elo_winner, elo_loser))*mov_multiplyer(elo_winner, elo_loser, score_winner, score_loser)

def post_elo_away_winner(elo_winner, elo_loser, score_winner, score_loser):
    return elo_winner + 20*(1-pr_winner_win(elo_winner, elo_loser))*mov_multiplyer(elo_winner, elo_loser, score_winner, score_loser)

In [3]:
# Import record of each team from 2018-19
from nbapy import game, shot_chart, player, scoreboard, team
east = scoreboard.Scoreboard(month=6, day=30, year=2019, league_id='00', offset=0).east_conf_standings_by_day()
east = east[["TEAM_ID", "TEAM", "W"]]
west = scoreboard.Scoreboard(month=6, day=30, year=2019, league_id='00', offset=0).west_conf_standings_by_day()
west = west[["TEAM_ID", "TEAM", "W"]]
standings = pd.concat([east, west])
standings

,TEAM_ID,TEAM,W
0,1610612749,Milwaukee,60
1,1610612761,Toronto,58
2,1610612755,Philadelphia,51
3,1610612738,Boston,49
4,1610612754,Indiana,48
5,1610612751,Brooklyn,42
6,1610612753,Orlando,42
7,1610612765,Detroit,41
8,1610612766,Charlotte,39
9,1610612748,Miami,39


In [4]:
# Estimate ELOs based on last season record
def elo_estimater(wins):
    return round((0.0029*(wins**3))-(0.3574*(wins**2))+(24.208*wins)+(907.97),2)

standings['ELO'] = [elo_estimater(x) for x in list(standings.W)]
standings = standings.reset_index(drop=1)
standings


,TEAM_ID,TEAM,W,ELO
0,1610612749,Milwaukee,60,1700.21
1,1610612761,Toronto,58,1675.57
2,1610612755,Philadelphia,51,1597.67
3,1610612738,Boston,49,1577.23
4,1610612754,Indiana,48,1567.22
5,1610612751,Brooklyn,42,1509.11
6,1610612753,Orlando,42,1509.11
7,1610612765,Detroit,41,1499.58
8,1610612766,Charlotte,39,1480.50
9,1610612748,Miami,39,1480.50


In [9]:
# Get games for 2019 into a table
games_2019 = pd.read_csv("../Resources/init_games.csv")
games_2019.date = games_2019.date.astype('datetime64')
games_2019.home_id = games_2019.home_id.astype('int64')
games_2019.away_id = games_2019.away_id.astype('int64')
games_2019 = games_2019[games_2019.home_id != 1610616834]
games_2019

,Unnamed: 0,date,game_id,home,away,home_id,away_id,home_pts,away_pts
0,0,2019-10-22,21900001,NOP,TOR,1610612740,1610612761,122.0,130.0
1,1,2019-10-22,21900002,LAL,LAC,1610612747,1610612746,102.0,112.0
2,2,2019-10-23,21900003,CHI,CHA,1610612741,1610612766,125.0,126.0
3,3,2019-10-23,21900004,DET,IND,1610612765,1610612754,119.0,110.0
4,4,2019-10-23,21900005,CLE,ORL,1610612739,1610612753,85.0,94.0
...,...,...,...,...,...,...,...,...,...
1135,1168,2020-10-02,41900402,MIA,LAL,1610612748,1610612747,114.0,124.0
1136,1169,2020-10-04,41900403,LAL,MIA,1610612747,1610612748,104.0,115.0
1137,1170,2020-10-06,41900404,LAL,MIA,1610612747,1610612748,102.0,96.0
1138,1171,2020-10-09,41900405,MIA,LAL,1610612748,1610612747,111.0,108.0


In [10]:
team_ids = list(standings['TEAM_ID'])
team_ids = [str(x) for x in team_ids]
last_season_ELOs = list(standings['ELO'])

season_start_elos = [round((0.75*x) + (0.25*1505), 2) for x in last_season_ELOs]

elos_2019 = pd.DataFrame([season_start_elos], columns=team_ids)
elos_2019


,1610612749,1610612761,1610612755,1610612738,1610612754,1610612751,1610612753,1610612765,1610612766,1610612748,...,1610612760,1610612746,1610612759,1610612758,1610612747,1610612750,1610612742,1610612763,1610612740,1610612756
0,1651.41,1632.93,1574.5,1559.17,1551.66,1508.08,1508.08,1500.93,1486.62,1486.62,...,1559.17,1551.66,1551.66,1486.62,1472.21,1464.93,1442.63,1442.63,1442.63,1320.34


In [11]:

dates_df = pd.DataFrame({'DATE':pd.date_range(start=games_2019.date.min(), end=games_2019.date.max()+timedelta(days=1))})

concat_df_labels = ['DATE']
for id in list(elos_2019):
    concat_df_labels.append(id)

elos_2019 = pd.concat([dates_df, elos_2019], ignore_index=1, axis=1)
elos_2019.columns = concat_df_labels

elos_2019


,DATE,1610612749,1610612761,1610612755,1610612738,1610612754,1610612751,1610612753,1610612765,1610612766,...,1610612760,1610612746,1610612759,1610612758,1610612747,1610612750,1610612742,1610612763,1610612740,1610612756
0,2019-10-22,1651.41,1632.93,1574.5,1559.17,1551.66,1508.08,1508.08,1500.93,1486.62,...,1559.17,1551.66,1551.66,1486.62,1472.21,1464.93,1442.63,1442.63,1442.63,1320.34
1,2019-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-10-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-10-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,2020-10-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,2020-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,2020-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,2020-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:

for index, game_row in games_2019.iterrows():
    # block1
    game_day = game_row.date
    home_score = game_row.home_pts
    away_score = game_row.away_pts

    if home_score > away_score:
        home_win = 1
    else:
        home_win = 0

    # print(game_day, home_score, away_score, home_win)

    # block2
    home_elo = [x for x in list(elos_2019[str(game_row.home_id)]) if x > 0][-1] + 100
    away_elo = [x for x in list(elos_2019[str(game_row.away_id)]) if x > 0][-1]

    # print(home_elo, away_elo)
    # block3
    if home_win == 1:
        # HOME WIN FUNCTION
        post_game_home_elo = round(post_elo_home_winner(home_elo, away_elo, home_score, away_score), 2)
        post_game_away_elo = round(away_elo + ((home_elo - 100) - post_game_home_elo), 2)
    elif home_win == 0:
        # AWAY WIN FUNCTION
        post_game_away_elo = round(post_elo_away_winner(away_elo, home_elo, away_score, home_score), 2)
        post_game_home_elo = round((home_elo - 100) + (away_elo - post_game_away_elo), 2)

    # print(post_game_home_elo, post_game_away_elo)
    next_day = game_day + timedelta(days=1)
    elos_2019.loc[list(elos_2019.DATE).index(next_day), str(game_row.home_id)] = post_game_home_elo
    elos_2019.loc[list(elos_2019.DATE).index(next_day), str(game_row.away_id)] = post_game_away_elo


In [15]:
elos_2019 = elos_2019.ffill(axis = 0)
elos_2019

,DATE,1610612749,1610612761,1610612755,1610612738,1610612754,1610612751,1610612753,1610612765,1610612766,...,1610612760,1610612746,1610612759,1610612758,1610612747,1610612750,1610612742,1610612763,1610612740,1610612756
0,2019-10-22,1651.41,1632.93,1574.50,1559.17,1551.66,1508.08,1508.08,1500.93,1486.62,...,1559.17,1551.66,1551.66,1486.62,1472.21,1464.93,1442.63,1442.63,1442.63,1320.34
1,2019-10-23,1651.41,1639.25,1574.50,1559.17,1551.66,1508.08,1508.08,1500.93,1486.62,...,1559.17,1562.83,1551.66,1486.62,1461.04,1464.93,1442.63,1442.63,1436.31,1320.34
2,2019-10-24,1651.41,1639.25,1591.59,1542.08,1543.61,1504.84,1514.93,1508.98,1490.14,...,1549.60,1562.83,1556.75,1442.03,1461.04,1468.17,1454.98,1423.44,1436.31,1364.93
3,2019-10-25,1655.29,1639.25,1591.59,1542.08,1543.61,1504.84,1514.93,1494.46,1490.14,...,1549.60,1576.46,1556.75,1442.03,1461.04,1468.17,1454.98,1423.44,1436.31,1364.93
4,2019-10-26,1655.29,1625.35,1591.59,1555.98,1543.61,1508.86,1514.93,1494.46,1477.30,...,1536.79,1576.46,1556.75,1438.58,1479.28,1481.01,1460.14,1415.12,1431.15,1362.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,2020-10-08,1626.96,1710.39,1510.45,1690.35,1522.07,1483.01,1464.25,1333.44,1360.82,...,1526.79,1642.45,1512.80,1479.91,1699.16,1372.58,1526.92,1515.82,1498.60,1580.39
353,2020-10-09,1626.96,1710.39,1510.45,1690.35,1522.07,1483.01,1464.25,1333.44,1360.82,...,1526.79,1642.45,1512.80,1479.91,1699.16,1372.58,1526.92,1515.82,1498.60,1580.39
354,2020-10-10,1626.96,1710.39,1510.45,1690.35,1522.07,1483.01,1464.25,1333.44,1360.82,...,1526.79,1642.45,1512.80,1479.91,1694.26,1372.58,1526.92,1515.82,1498.60,1580.39
355,2020-10-11,1626.96,1710.39,1510.45,1690.35,1522.07,1483.01,1464.25,1333.44,1360.82,...,1526.79,1642.45,1512.80,1479.91,1694.26,1372.58,1526.92,1515.82,1498.60,1580.39


In [22]:
list(elos_2019), list(elos_2019.iloc[-1])

(['DATE',
  '1610612749',
  '1610612761',
  '1610612755',
  '1610612738',
  '1610612754',
  '1610612751',
  '1610612753',
  '1610612765',
  '1610612766',
  '1610612748',
  '1610612764',
  '1610612737',
  '1610612741',
  '1610612739',
  '1610612752',
  '1610612744',
  '1610612743',
  '1610612745',
  '1610612757',
  '1610612762',
  '1610612760',
  '1610612746',
  '1610612759',
  '1610612758',
  '1610612747',
  '1610612750',
  '1610612742',
  '1610612763',
  '1610612740',
  '1610612756'],
 [Timestamp('2020-10-12 00:00:00'),
  1626.96,
  1710.39,
  1510.45,
  1690.35,
  1522.07,
  1483.01,
  1464.25,
  1333.44,
  1360.82,
  1632.54,
  1364.76,
  1357.67,
  1345.09,
  1345.11,
  1335.26,
  1347.12,
  1571.9,
  1548.83,
  1508.9,
  1564.62,
  1526.79,
  1642.45,
  1512.8,
  1479.91,
  1700.58,
  1372.58,
  1526.92,
  1515.82,
  1498.6,
  1580.39])